In [2]:
import tkinter as tk
import tkinter.messagebox as msgbox
import tkinter.font as font
from tkinter import filedialog

from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [3]:
class Scraping():
    
    def __init__(self, url):
        self.url = url
        
        title = []
        location = []
        company_name = []
        job_req = []
        job_des = []

        req = requests.get(url)

        if str(req) == '<Response [200]>':
            soup = bs(req.content, 'html.parser')
            lst = soup.find_all('a', {'class':'base-card__full-link'})

            links = []
            for i in lst:
                links.append(i['href'])

            for i in range(len(links)):
                test_req = requests.get(links[i])
                if str(test_req) == '<Response [200]>':
                    soup_ = bs(test_req.content, 'html.parser')

                    #title
                    title.append(soup_.find_all('h1')[0].text)

                    #location
                    location.append(soup_.find_all('span', class_='sub-nav-cta__meta-text')[0].text)

                    #company name
                    company_name.append(soup_.find_all('span')[6].find('a').text.strip())

                    #job requriments
                    job_req.append([i.text for i in soup_.find_all('ul')[5].find_all('li')])

                    #job description
                    job_des.append([i.text for i in soup_.find_all('ul')[4].find_all('li')])


        df = pd.DataFrame({'title':title, 'company_name':company_name, 'job_description':job_des,
                 'job_requirments':job_req, 'location':location})

        return df
    
    
    def create_files(df):
        for i in range(len(df)):
            ind = df.iloc[i, :].index
            value = df.iloc[i, :].values
            dic = {x:y  for x, y in zip(ind, value)}
            tit = df['title'][i]+'.txt'
            tit = tit.replace('/','').replace('|', '')

            with open(tit, 'w', encoding='utf-8') as f:
                for col in ind:
                    f.writelines(col+'\n'+ dic[col]+'\n\n') 


In [4]:
class Card:
    def __init__(self, suit, value):
        self.suit = suit
        self.value = value
        
    def __repr__(self):
        return " of ".join((self.value, self.suit))

In [6]:
window = tk.Tk()
window.title('Scraping')
        
window.geometry("1000x700")
        
labelFont = font.Font(family='Courier', size=15)
t1 = tk.Label(text="Enter URL you want to scarp", fg='red', font=labelFont)
t1.pack(padx=20, pady=20)
        
entry = tk.Entry()
entry.pack()
        
def submitValues():
    url = entry.get() 
    card = Card(url, 'A5')
#     data = Scraping(url)
#     data.to_csv('Try.csv', index=False)
    tk.Label(text='Done sucussfully.You can check the current directory, you will find the data file\nWe will load analysis later').pack()
    
    options = {}
    options['filetypes'] = [
                    ('csv files', '.csv')
                        ]
    options['title'] = 'Choose a file'

    filedialog.askopenfile(**options)

        
submit = tk.Button(text="Submit", command=submitValues)
submit.pack()


window.mainloop()

In [ ]:
https://www.linkedin.com/jobs/search/?keywords=data%20science%20jobs&location=&refresh=true